# 1. Fetching the relevant information in JSON format

In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature = 0,
    api_key = "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" ,
    model = "llama-3.3-70b-versatile"
)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader("https://jobs.apple.com/en-in/details/200594546/machine-learning-engineer-data-quality?team=OPMFG")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.











Machine Learning Engineer - Data Quality - Jobs at Apple (IN)







AppleStoreMaciPadiPhoneWatchAirPodsTV & HomeEntertainmentAccessoriesSupport


0+
Jobs at AppleOpen MenuClose Menu

      Work at Apple
    
 

      Life at Apple
    
 

      Profile
    
 

      Sign In
    
 
Search
Jobs at Apple
Machine Learning Engineer - Data QualityBengaluru, Karnataka, IndiaOperations and Supply ChainAdd to Favourites Machine Learning Engineer - Data QualityRemoved from favouritesAdd a favoriteCloseTo view your favourites, sign in with your Apple Account.Sign InDon’t have an Apple Account?Create one nowForgot your Apple Account or password?Submit CVMachine Learning Engineer - Data QualityBack to search resultsSummaryPosted: 11 Mar 2025Role Number:200594546Join Apple’s Data Management Team and Shape the Future of Reliable Data in the Manufacturing Space!

At Apple, we believe that great decisions start with great data. Our Data Quality team within Operations is looking for a Machine 

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """ SCRAP TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTIONS:
    The scraped text is from the career's pafe of a website.
    Your job is to extract the job postings and return them in JSON format containing
    following keys: 'role', 'experience', 'skills', and 'descriptions'.
    Only return the valid JSON.
    ## VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res
job = json_res
job['skills']

['Python (Pandas, NumPy, Scikit-learn, PyTorch/TensorFlow, etc.)',
 'SQL and databases (PostgreSQL, Snowflake, MySQL, etc.)',
 'Statistical methods (PCA, exponential smoothing, etc.)',
 'Version control (Git) and software development best practices',
 'MLOps tools (MLflow, Kubeflow)',
 'Big data frameworks (Spark, Kafka)',
 'CI/CD for data pipelines and model deployments']

# 2. Creating a ChromaDB for the content

In [20]:
import pandas as pd

df = pd.read_csv("C:/Users/Stuti/Downloads/vscode/.vscode/Practise/Llama3.1_gen/app/resource/my_portfolio.csv")

In [14]:
import chromadb
import uuid

client =  chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row['Techstack'],
                       metadatas={'links': row['Links']},
                       ids = [str(uuid.uuid4())])  

In [15]:
links = collection.query(
    query_texts = ['skills'] ,
    n_results =  2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Stuti, an Artificial Intelligence Engineer at Frontera. Frontera is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Frontera 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Frontera's portfolio: {link_list}
        Remember you are Mohan, BDE at Frontera. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res_email = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res_email)

content='Subject: Expert Machine Learning Solutions for Data Quality Enhancement\n\nDear Hiring Manager,\n\nI came across the job description for a Machine Learning Engineer - Data Quality, and I am excited to introduce Frontera, an AI & Software Consulting company that can fulfill your requirements. With 3+ years of experience in building ML solutions and collaborating with software teams, our team of experts can develop ML-based data validation and monitoring solutions, focusing on anomaly detection and explainability.\n\nAt Frontera, we have a strong portfolio in machine learning and data science, with expertise in Python (Pandas, NumPy, Scikit-learn, PyTorch/TensorFlow, etc.), SQL and databases (PostgreSQL, Snowflake, MySQL, etc.), and statistical methods (PCA, exponential smoothing, etc.). Our team is well-versed in version control (Git) and software development best practices, ensuring seamless collaboration and delivery.\n\nWe have experience in applying the full ML lifecycle, f